url : https://www.kaggle.com/datasets/henriqueyamahata/bank-marketing                  
downlaod : https://www.kaggle.com/datasets/henriqueyamahata/bank-marketing/download?datasetVersionNumber=1

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import plotly.express as px

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.svm import SVC

In [17]:
data = pd.read_csv('/kaggle/input/bank-marketing/bank-additional-full.csv',delimiter=';')

In [29]:
data

,age,default,housing,loan,month,day_of_week,duration,campaign,pdays,previous,...,JOB__unemployed,JOB__unknown,ED__basic.4y,ED__basic.6y,ED__basic.9y,ED__high.school,ED__illiterate,ED__professional.course,ED__university.degree,ED__unknown
0,56,0,0,0,6,1,261,1,999,0,...,0,0,1,0,0,0,0,0,0,0
1,57,0,0,0,6,1,149,1,999,0,...,0,0,0,0,0,1,0,0,0,0
2,37,0,1,0,6,1,226,1,999,0,...,0,0,0,0,0,1,0,0,0,0
3,40,0,0,0,6,1,151,1,999,0,...,0,0,0,1,0,0,0,0,0,0
4,56,0,0,1,6,1,307,1,999,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,0,1,0,7,0,334,1,999,0,...,0,0,0,0,0,0,0,1,0,0
41184,46,0,0,0,7,0,383,1,999,0,...,0,0,0,0,0,0,0,1,0,0
41185,56,0,1,0,7,0,189,2,999,0,...,0,0,0,0,0,0,0,0,1,0
41186,44,0,0,0,7,0,442,1,999,0,...,0,0,0,0,0,0,0,1,0,0


In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

In [20]:
data.isnull().sum().sum()

0

## Encoding

In [30]:
{col : data[col].unique() for col in data.columns if data[col].dtypes == 'object'}

{}

In [22]:
need_help_bin = ['default','housing','loan','y']
for column in need_help_bin:
    data[column] = data[column].apply(lambda x: 1 if x=='yes' else 0)

In [23]:
ordinal_1 = sorted(['may', 'jun', 'jul', 'aug', 'oct', 'nov', 'dec', 'mar', 'apr', 'sep'])
ordinal_2 = sorted(['nonexistent', 'failure', 'success'])
ordinal_3 = sorted(['mon', 'tue', 'wed', 'thu', 'fri'])
data['month'] = data['month'].apply(lambda x: ordinal_1.index(x))
data['poutcome'] = data['poutcome'].apply(lambda x: ordinal_2.index(x))
data['day_of_week'] = data['day_of_week'].apply(lambda x: ordinal_3.index(x))

In [28]:
nominal_fet = ['marital','contact','job','education']
prefixes = ['MAT_','CON_','JOB_','ED_']
for cols , prefix in zip(nominal_fet,prefixes):
    dummie = pd.get_dummies(data[cols],prefix=prefix).astype(np.int)
    data = pd.concat([data,dummie],axis=1)
    data = data.drop(cols , axis=1)

/tmp/ipykernel_32/4122361990.py:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dummie = pd.get_dummies(data[cols],prefix=prefix).astype(np.int)
/tmp/ipykernel_32/4122361990.py:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
D

In [32]:
y = data['y']
x = data.drop('y',axis=1)

In [37]:
#y.mean()
#x.shape

(41188, 42)

In [38]:
scaler = StandardScaler()
x = scaler.fit_transform(x)

In [39]:
x_train , x_test , y_train , y_test = train_test_split(x,y,train_size=0.7,random_state=7)

In [52]:
inputs = tf.keras.Input(shape=(42,))
x = tf.keras.layers.Dense(128,activation='relu')(inputs)
x = tf.keras.layers.Dense(64,activation='relu')(x)
outputs = x = tf.keras.layers.Dense(1,activation='sigmoid')(x)

model = tf.keras.Model(inputs=inputs , outputs=outputs)

model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics=[tf.keras.metrics.AUC(name='auc')]
)
epochs = 5

hist = model.fit(
    x_train,
    y_train,
    validation_split = 0.2,
    batch_size = 64,
    epochs=epochs,
    callbacks=tf.keras.callbacks.ReduceLROnPlateau()
)

Epoch 1/5
361/361 [==============================] - 2s 3ms/step - loss: 0.2297 - auc: 0.8987 - val_loss: 0.1994 - val_auc: 0.9302 - lr: 0.0010
Epoch 2/5
361/361 [==============================] - 1s 2ms/step - loss: 0.1959 - auc: 0.9313 - val_loss: 0.1968 - val_auc: 0.9312 - lr: 0.0010
Epoch 3/5
361/361 [==============================] - 1s 2ms/step - loss: 0.1899 - auc: 0.9355 - val_loss: 0.1933 - val_auc: 0.9345 - lr: 0.0010
Epoch 4/5
361/361 [==============================] - 1s 2ms/step - loss: 0.1843 - auc: 0.9404 - val_loss: 0.1920 - val_auc: 0.9349 - lr: 0.0010
Epoch 5/5
361/361 [==============================] - 1s 2ms/step - loss: 0.1805 - auc: 0.9427 - val_loss: 0.1959 - val_auc: 0.9319 - lr: 0.0010


In [53]:
fig = px.line(
    hist.history,
    y=['loss','val_loss'],
    labels={'x' : 'Epochs' , 'y':'Loss'},
    title = 'training and validation loss'
)
fig.show()

In [54]:
np.argmin(hist.history['val_loss'])+1

4

In [55]:
model.evaluate(x_test,y_test)

387/387 [==============================] - 1s 1ms/step - loss: 0.1925 - auc: 0.9308


[0.19247794151306152, 0.9307659268379211]

### Thank you so much for watching 
##### **i hope you enjoy:)**